In [54]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.8 ms


In [55]:
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os


from dev import APP_DATA_DIR, NAME_COLS, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, PICKLE_DIR

from data_cleaning_functions import has_suffix, remove_suffix_from_last_name 

time: 5.56 ms


In [80]:
filename = 'fuzzy_all_apps_plus_NIH_info.csv'

time: 1.7 ms


In [93]:
apps = pd.read_csv(os.path.join(APP_DATA_DIR, filename))

time: 77.2 ms


In [94]:
# for two people whom I manually added names for, fill in first and middle and last names
mask = pd.isnull(apps.last_name)
apps.loc[mask, 'last_name'] = apps[mask].clean_last_name.apply(lambda x: x.title())
mask = pd.isnull(apps.middle_name)
apps.loc[mask, 'middle_name'] = apps[mask].clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else x.title())
mask = pd.isnull(apps.first_name)
apps.loc[mask, 'first_name'] = apps[mask].clean_first_name.apply(lambda x: np.nan if pd.isnull(x) else x.title())

time: 23.4 ms


In [95]:
# generate a file with first initial of first name, and last name
# also generate a row with middle initial, and last name
# check for last names with spaces in them, mc last names, last names with an apostrophe
apps['first_initial'] = apps.clean_first_name.apply(lambda x: np.nan if pd.isnull(x) else str(x[0]))
apps['middle_initial'] = apps.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else str(x[0]))
apps['clean_last_name2'] = apps.last_name.apply(lambda x: np.nan if pd.isnull(x) else x.upper().strip())
apps['clean_last_name3'] = np.nan

time: 34.6 ms


In [96]:
suff_mask = apps.clean_last_name.apply(has_suffix)

# remove suffixes from their last name
# also remove any commas from the last name
apps.loc[suff_mask, 'clean_last_name2'] = apps[suff_mask]['clean_last_name2'].apply(remove_suffix_from_last_name)

time: 31.6 ms


In [97]:
def strip_commas(raw_str):
    if pd.isnull(raw_str):
        return np.nan
    punc = ('.', ',')
    return ''.join([x for x in raw_str if x not in punc])

apps['clean_last_name2'] = apps['clean_last_name2'].apply(strip_commas)

time: 29 ms


In [92]:
suff_mask = apps.clean_last_name.apply(has_suffix)

# remove suffixes from their last name
# also remove any commas from the last name
apps.loc[suff_mask, 'clean_last_name2'] = apps[suff_mask]['clean_last_name2'].apply(remove_suffix_from_last_name)

time: 23.9 ms


In [98]:
def find_space_in_name(raw_str):
    # finds strings with spaces in them
    if pd.isnull(raw_str):
        return False
    str_list = raw_str.split(' ')
    return len(str_list) >= 2

time: 7.38 ms


In [99]:
# for last names with a space in them, create version with and without space
odd_last_name = apps.clean_last_name.apply(find_space_in_name)

apps.loc[odd_last_name, 'clean_last_name3'] = apps[
    odd_last_name]['clean_last_name'].apply(lambda x: ''.join(x.split(' ')).strip())

apps.loc[apps.clean_last_name==apps.clean_last_name2, 'clean_last_name2'] = np.nan

mask = (pd.isnull(apps.clean_last_name2)) & ~(pd.isnull(apps.clean_last_name3))
apps.loc[mask, 'clean_last_name2'] = apps[mask]['clean_last_name3']

apps.loc[mask, 'clean_last_name3'] = np.nan
apps.loc[apps.clean_last_name==apps.clean_last_name2, 'clean_last_name2'] = np.nan

time: 37 ms


In [100]:
def make_hash_str(row):
    strs = row[['first_name', 'last_name', 'middle_name', 'address', 'original_medical_school']].values
    
    str_val = ''.join(map(str, strs))
    return uuid.uuid5(uuid.NAMESPACE_DNS, str_val)

time: 4.28 ms


In [101]:
apps['hash_id'] = apps.apply(make_hash_str, axis=1)

time: 1.71 s


In [102]:
apps.loc[apps.duplicated('hash_id', keep=False), NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name


time: 16.6 ms


In [123]:
apps['first_name2'] = apps.first_name.apply(lambda x: np.nan if pd.isnull(x) else x.upper().strip())
apps['first_initial2'] = apps.first_name2.apply(lambda x: np.nan if pd.isnull(x) else x[0])
apps['middle_name2'] = apps.middle_name.apply(lambda x: np.nan if pd.isnull(x) else x.upper().strip())
apps['middle_initial2'] = apps.middle_name2.apply(lambda x: np.nan if pd.isnull(x) else x[0])
apps.loc[apps.first_initial==apps.first_initial2, 'first_initial2'] = np.nan
mask = (pd.isnull(apps.middle_initial) & ~pd.isnull(apps.middle_initial2))
apps.loc[mask, 'middle_initial'] = apps[mask]['middle_initial2']
apps.loc[apps.middle_initial==apps.middle_initial2, 'middle_initial2'] = np.nan
apps.loc[apps.middle_initial.isin(['1', '*']), 'middle_initial'] = np.nan
apps.loc[apps.middle_initial2.isin(['1', '*']), 'middle_initial2'] = np.nan

time: 31.7 ms


In [124]:
apps.loc[apps.duplicated('hash_id', keep=False), NAME_COLS+[
        'dno', 'address', 'original_medical_school', 'med_school', 'hash_str', 'first_name', 'middle_name', 'last_name']]

,clean_first_name,clean_middle_name,clean_last_name,dno,address,original_medical_school,med_school,hash_str,first_name,middle_name,last_name


time: 23.4 ms


In [136]:
# all first initials are from first name data entry errors
n1_ = apps.loc[~pd.isnull(apps.first_initial2)]
n1 = apps.loc[~pd.isnull(apps.middle_initial2),]

time: 7.31 ms


In [137]:
n1.loc[:, 'middle_initial'] = n1['middle_initial2']
del n1['middle_initial2']

time: 35.3 ms


/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [147]:
n2 = apps.loc[~(pd.isnull(apps.clean_last_name2)), :]
n2.loc[:, 'clean_last_name'] = n2['clean_last_name2']
del n2['clean_last_name2']
n3 = apps.loc[~(pd.isnull(apps.clean_last_name3)), :]
n3.loc[:, 'clean_last_name'] = n3['clean_last_name2']
del n3['clean_last_name3']


time: 49.9 ms


In [148]:
cols = [
    'first_name', 'middle_name', 'last_name', 'dno', 'hash_id', 'person_uuid', 'first_initial', 
    'middle_initial', 'clean_last_name', 'medical_school', 'application_year_min', 'application_year_max'
] + NAME_COLS

time: 2.79 ms


In [149]:
# concat all the rows together
all_versions = pd.concat([apps, n1, n2, n3], axis=0).sort_values(NAME_COLS+['hash_id'])

time: 109 ms


In [150]:
# to csv 
all_versions[cols].to_csv(os.path.join(APP_DATA_DIR, 'multi_name_versions_AMA_match.csv'), index=False)

time: 635 ms
